In [1]:
import pandas as pd
import numpy as np
import polars as pl
import datetime

pd.set_option("display.max_columns", None)

In [2]:
### Target col
[
    "hospcode",
    "hosname",
    "name_th",
    "county",
    "changwatname",
    "ampurname",
    "tambonname",
    "sex",
    "birth",
    "age",
    "nation",
    "typearea",
    "diagcode",
    "diagtype",
    "date_serv",
    "chiefcomp",
    "occupation",
    "diagname",
    "typediag",
]
### Current_col
current_col = [
    "opd.hospcode",
    "c.hosname",
    "opd.pid",
    "opd.seq",
    "p.cid",
    "p.sex",
    "p.birth",
    "age",
    "p.nation",
    "p.typearea",
    "opd.diagcode",
    "opd.diagtype",
    "opd.date_serv",
    "p.occupation_new",
    "occ.occupation_new",
    "p.occupation_old",
    "old.occupation",
    "c.provcode",
    "ch.changwatname",
    "c.distcode",
    "ca.ampurname",
    "c.subdistcode",
    "ct.tambonname",
    "s.instype",
    "s.typeout",
    "s.chiefcomp",
]

In [3]:
occ = pl.read_excel("./utilities/occupations.xlsx", schema_overrides={"code": str})
hoscode = pl.read_csv("./utilities/hospcode.csv", dtypes={"hospcode": str}).select(
    [
        "hospcode",
        "name_th",
        pl.col("changwatname").alias("province"),
        pl.col("ampurname").alias("district"),
        pl.col("tambonname").alias("subdistrict"),
    ]
)
total_pop = (
    pl.read_csv("./utilities/total_pop.csv", dtypes={"year": str})
    .select(["year", "total_pop", "DistrictThaiShort", "ProvinceThai"])
    .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
    .agg(pl.col("total_pop").sum())
)

icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": str}).unique()

nation = pl.read_csv("./utilities/nation.csv", dtypes={"code": str}).vstack(
    pl.DataFrame({"code": ["99", "57", "48"], "nation": ["ไทย", "กัมพูชา", "เมียนมา"]})
)

odpc = (
    pl.read_excel("./utilities/Template_เขตสุขภาพ_ภูมิภาค.xlsx")
    .with_columns(pl.col("Regional Health").str.strip_prefix("เขตสุขภาพที่ "))
    .select(
        [pl.col("จังหวัด").alias("province"), pl.col("Regional Health").alias("odpc")]
    )
)

C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2639653350.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  hoscode = pl.read_csv("./utilities/hospcode.csv", dtypes={"hospcode": str}).select(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2639653350.py:12: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv("./utilities/total_pop.csv", dtypes={"year": str})
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2639653350.py:14: DeprecationWarning: named `by` param is deprecated; use positional `*args` instead.
  .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2639653350.py:18: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": str}).unique()
C

In [4]:
data = (
    pl.read_csv(
        "../output_csv/",
        has_header=False,
        new_columns=current_col,
        dtypes={
            "column_1": str,
            "column_4": str,
            "column_24": str,
            "column_3": str,
            "column_14": str,
            "column_12": str,
            "column_25": str,
            "column_9": str,
        },
    )
    .drop_nulls(subset=["p.cid"])
    .drop(
        columns=[
            "c.hosname",
            "c.provcode",
            "ch.changwatname",
            "c.distcode",
            "ca.ampurname",
            "c.subdistcode",
            "ct.tambonname",
        ]
    )
    .filter(
        (~pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา|มาตามนัด"))
        | (
            (pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา"))
            & pl.col("s.chiefcomp").str.contains("มีอาการ")
        )
    )
    .join(hoscode, how="left", left_on="opd.hospcode", right_on="hospcode")
)

C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2464797863.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2464797863.py:18: DeprecationWarning: named `columns` param is deprecated; use positional `*args` instead.
  .drop(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\2464797863.py:36: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(hoscode, how="left", left_on="opd.hospcode", right_on="hospcode")


In [5]:
data

opd.hospcode,opd.pid,opd.seq,p.cid,p.sex,p.birth,age,p.nation,p.typearea,opd.diagcode,opd.diagtype,opd.date_serv,p.occupation_new,occ.occupation_new,p.occupation_old,old.occupation,s.instype,s.typeout,s.chiefcomp,name_th,province,district,subdistrict
str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""08954""","""15826""","""467930""","""e2cdc1587dbc012c168e401d120550…",1,"""2010-10-09""",12,"""099""",4,"""H103""","""1""","""2023-09-07""","""9999""","""ไม่มีงานทำ""","""900""","""หญิงบริการ""","""0100""","""9""","""เจ็บตาข้างขวา เป็นมา3วัน""","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""นครศรีธรรมราช""","""บางขัน""","""วังหิน"""
"""10688""","""001021""","""4595032""","""bac9dae52ed06f613c9cc0d7ebe0bf…",2,"""1963-11-01""",59,"""099""",4,"""H109""","""1""","""2023-06-26""","""9005""","""ลูกจ้างชั่วคราว""","""209""",null,"""4200""","""1""","""มีอาการระคายเคืองตา คัน""","""โรงพยาบาลเสนา""","""พระนครศรีอยุธยา""","""เสนา""","""เจ้าเจ็ด"""
"""10688""","""016221""","""4546922""","""fdc41e84d4c24927f377ba387a2c0a…",2,"""1976-06-01""",46,"""099""",4,"""H109""","""1""","""2023-04-19""","""9622""","""คนงานรับจ้างทั่วไป""","""403""",null,"""4200""","""1""","""3 วัน เจ็บตาซ้ายหลังตื่นนอน""","""โรงพยาบาลเสนา""","""พระนครศรีอยุธยา""","""เสนา""","""เจ้าเจ็ด"""
"""10688""","""245881""","""4652262""","""cb828385cfb3aa30dfb3ccc4b097d5…",2,"""1990-07-25""",33,"""099""",4,"""H100""","""1""","""2023-09-12""","""1112""","""ข้าราชการระดับอาวุโส : ผุ้ปฏิบ…","""201""",null,"""1100""","""1""","""ขอยาหยอดตา""","""โรงพยาบาลเสนา""","""พระนครศรีอยุธยา""","""เสนา""","""เจ้าเจ็ด"""
"""10754""","""923898""","""2644372""","""790f57944be65cf2f2a62b83af1a64…",2,"""2003-07-11""",19,"""099""",4,"""H103""","""4""","""2023-03-15""","""9999""","""ไม่มีงานทำ""","""899""",null,"""0100""","""1""","""ปวดเคืองตาขวา 1ชม ผู้ป่วยให้ปร…","""โรงพยาบาลบางจาก""","""สมุทรปราการ""","""พระประแดง""","""เทศบาลเมืองพระประแดง"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""01623""","""003918""","""157891""","""6fee8b8c75cc3f9cab1311a6cdfcc6…",1,"""1981-08-29""",41,"""099""",1,"""H101""","""1""","""2023-01-09""","""9629""","""ผู้ประกอบวิชาชีพงานพื้นฐานอื่น…","""902""",null,"""1100""","""1""","""ปวด เคืองตา ร้าว ในตา น้ำตาไหล…","""โรงพยาบาลส่งเสริมสุขภาพตำบลไร่…","""ชัยนาท""","""มโนรมย์""","""ไร่พัฒนา"""
"""11321""","""010586""","""066090575""","""15febfdc6e57fbee055f399fdcf45d…",2,"""1960-06-08""",63,"""099""",1,"""H100""","""1""","""2023-06-20""","""6111""","""ผู้ปลูกพืชไร่และพืชผัก, ทำไร่,…","""001""","""เกษตรกรรม""","""0100""","""1""","""แสบตาสองข้าง เป็นมา6 วัน""","""โรงพยาบาลสามร้อยยอด""","""ประจวบคีรีขันธ์""","""สามร้อยยอด""","""ไร่ใหม่"""
"""08609""","""6798""","""115778""","""2039b9ae6b17775854bdf931543944…",2,"""1992-04-01""",31,"""099""",4,"""H103""","""1""","""2023-11-04""","""9622""","""คนงานรับจ้างทั่วไป""","""002""","""รับจ้าง""","""0100""","""9""","""เคืองตา ตาแดงเป็นมา 1 วัน""","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""เพชรบุรี""","""ชะอำ""","""ไร่ใหม่พัฒนา"""


In [6]:
data = data.to_pandas()
data.loc[data["opd.hospcode"] == "99782", "name_th"] = (
    "ศูนย์สุขภาพชุมชนโรงพยาบาลสทิงพระ(สทิงพระ)"
)
data.loc[data["opd.hospcode"] == "99782", "province"] = "สงขลา"
data.loc[data["opd.hospcode"] == "99782", "district"] = "สทิงพระ"
data.loc[data["opd.hospcode"] == "99782", "subdistrict"] = "จะทิ้งพระ"

data.loc[data["opd.hospcode"] == "99942", "name_th"] = "ศูนย์สุขภาพชุมชนโรงพยาบาลบึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "province"] = "บึงกาฬ"
data.loc[data["opd.hospcode"] == "99942", "district"] = "บึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "subdistrict"] = "บึงโขงหลง"

data.loc[data["opd.hospcode"] == "99742", "name_th"] = (
    "หน่วยบริการปฐมภูมิ กลุ่มงานเวชปฏิบัติครอบครัวและชุมชน รพ.บันนังสตา"
)
data.loc[data["opd.hospcode"] == "99742", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99742", "district"] = "บันนังสตา"
data.loc[data["opd.hospcode"] == "99742", "subdistrict"] = "บันนังสตา"

data.loc[data["opd.hospcode"] == "99745", "name_th"] = (
    "ศูนย์สุขภาพชุมชนเมืองเวชกรรม (เมืองยะลา)"
)
data.loc[data["opd.hospcode"] == "99745", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99745", "district"] = "เมือง"
data.loc[data["opd.hospcode"] == "99745", "subdistrict"] = "สะเตง"

data.loc[data["opd.hospcode"] == "99763", "name_th"] = "ศูนย์สุขภาพชุมชนตำบลยามู"
data.loc[data["opd.hospcode"] == "99763", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99763", "district"] = "ยะหริ่ง"
data.loc[data["opd.hospcode"] == "99763", "subdistrict"] = "ยามู"


data.loc[data["name_th"].isnull(), "name_th"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "province"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "district"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "subdistrict"] = "ไม่ระบุ"

data.loc[data["opd.hospcode"] == "99874", "name_th"] = "ศุนย์สุขภาพชุมชนโรงพยาบาลยะรัง"
data.loc[data["opd.hospcode"] == "99874", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99874", "district"] = "ยะรัง"
data.loc[data["opd.hospcode"] == "99874", "subdistrict"] = "ปิตูมุดี"
data = pl.from_pandas(data)

In [7]:
data_2 = (
    data.with_columns(
        pl.when(pl.col("occ.occupation_new").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("occ.occupation_new"))
        .alias("occ.occupation_new")
    )
    .with_columns(
        pl.when(pl.col("p.sex") == 1)
        .then(pl.lit("ชาย"))
        .when(pl.col("p.sex") == 2)
        .then(pl.lit("หญิง"))
        .otherwise(pl.lit("ไม่ระบุ"))
        .alias("p.sex")
    )
    .with_columns(pl.col("opd.diagcode").str.replace("[.|-]", ""))
    .with_columns(pl.col("opd.diagcode").str.to_uppercase())
    .with_columns(
        pl.when(pl.col("opd.diagtype") == "1")
        .then(pl.lit("principal dx"))
        .when(pl.col("opd.diagtype") == "2")
        .then(pl.lit("co-morbidity"))
        .when(pl.col("opd.diagtype") == "3")
        .then(pl.lit("complication"))
        .when(pl.col("opd.diagtype") == "4")
        .then(pl.lit("other"))
        .when(pl.col("opd.diagtype") == "5")
        .then(pl.lit("external"))
        .when(pl.col("opd.diagtype") == "6")
        .then(pl.lit("additional"))
        .when(pl.col("opd.diagtype") == "0")
        .then(pl.lit("principal dx"))
        .otherwise(pl.lit("morph"))
        .alias("opd.diagtype")
    )
    .with_columns(dx_group=pl.lit("กลุ่มโรคตาอักเสบ"))
    .with_columns(
        pl.col("opd.date_serv")
        .str.to_date("%Y-%m-%d")
        .dt.strftime("%d-%m-%Y")
        .alias("opd.date_serv")
    )
    .join(icd10, how="left", left_on="opd.diagcode", right_on="code")
    .join(nation, how="left", left_on="p.nation", right_on="code")
    .join(odpc, how="left", left_on="province", right_on="province")
    .group_by(["p.cid", "opd.date_serv"])
    .first()
    .select(
        [
            pl.col("opd.date_serv").alias("visit_date"),
            pl.col("odpc"),
            pl.col("province"),
            pl.col("district"),
            pl.col("subdistrict"),
            pl.col("opd.hospcode").alias("hospcode"),
            pl.col("name_th").alias("hosname"),
            pl.col("p.sex").alias("sex"),
            pl.col("p.birth").alias("birth"),
            pl.col("age"),
            pl.col("nation"),
            pl.col("s.chiefcomp"),
            pl.col("occ.occupation_new").alias("occ"),
            pl.col("p.typearea").alias("typearea"),
            pl.col("opd.diagcode").alias("dx_code"),
            pl.col("opd.diagtype").alias("dx_type"),
            pl.col("dx_group"),
            pl.col("dz_name"),
        ]
    )
    .with_columns(
        pl.when(pl.col("nation").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("nation"))
        .alias("nation")
    )
    .with_row_count(name="id", offset=1)
    .with_columns(create_date=datetime.date.today())
)

C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\1148687097.py:43: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(icd10, how="left", left_on="opd.diagcode", right_on="code")
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\1148687097.py:44: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(nation, how="left", left_on="p.nation", right_on="code")
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_20136\1148687097.py:45: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(odpc, how="left", left_on="province", right_on="province")
C:\Users\Rawin

In [8]:
data_2

id,visit_date,odpc,province,district,subdistrict,hospcode,hosname,sex,birth,age,nation,s.chiefcomp,occ,typearea,dx_code,dx_type,dx_group,dz_name,create_date
u32,str,str,str,str,str,str,str,str,str,i64,str,str,str,f64,str,str,str,str,date
1,"""19-10-2023""","""1""","""เชียงใหม่""","""เมืองเชียงใหม่""","""วัดเกต""","""11508""","""โรงพยาบาลค่ายกาวิละ เชียงใหม่""","""หญิง""","""1957-02-19""",66,"""ไทย""","""��Ǩ�ҵ���Ѵ�Ф����ͧ�ҵҢ���բ����ҡ��…","""ข้าราชการการเมืองและนักการเมือ…",4.0,"""H101""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Acute atopic conjunctivitis""",2024-06-13
2,"""02-10-2023""","""6""","""สมุทรปราการ""","""พระสมุทรเจดีย์""","""ในคลองบางปลากด""","""00985""","""โรงพยาบาลส่งเสริมสุขภาพตำบลในค…","""หญิง""","""1965-12-19""",57,"""ไทย""","""ตาแดง เป็นมา 1 วัน""","""คนงานรับจ้างทั่วไป""",1.0,"""H103""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Unspecified acute conjunctivit…",2024-06-13
3,"""30-01-2023""","""9""","""นครราชสีมา""","""เมืองนครราชสีมา""","""โคกกรวด""","""23839""","""โรงพยาบาลเทพรัตน์นครราชสีมา ""","""หญิง""","""1969-03-23""",53,"""ไทย""","""เคืองตาข้างขวา 16 ชั่วโมงก่อน…","""คนงานรับจ้างทั่วไป""",4.0,"""H103""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Unspecified acute conjunctivit…",2024-06-13
4,"""27-09-2023""","""4""","""นครนายก""","""เมืองนครนายก""","""นครนายก""","""10698""","""โรงพยาบาลนครนายก""","""หญิง""","""1955-09-27""",68,"""ไทย""","""มีจุดดำด้านในตาขวาเคืองเป็นเมื…","""ในปกครอง""",4.0,"""H101""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Acute atopic conjunctivitis""",2024-06-13
5,"""04-08-2023""","""7""","""ขอนแก่น""","""ซำสูง""","""กระนวน""","""14132""","""โรงพยาบาลซำสูง""","""ชาย""","""1969-07-01""",54,"""ไทย""","""case FB sensation Rt. eye มาติ…","""ผู้ประกอบวิชาชีพด้านศาสนา""",4.0,"""H103""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Unspecified acute conjunctivit…",2024-06-13
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
977431,"""13-03-2023""","""12""","""ตรัง""","""ห้วยยอด""","""ลำภูรา""","""09742""","""โรงพยาบาลส่งเสริมสุขภาพตำบลลำภ…","""หญิง""","""1935-08-25""",87,"""ไทย""","""1 วัน ท้องอืด ระคายเคืองตา""","""ไม่มีงานทำ""",1.0,"""H101""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Acute atopic conjunctivitis""",2024-06-13
977432,"""20-05-2023""","""7""","""กาฬสินธุ์""","""หนองกุงศรี""","""หนองกุงศรี""","""11086""","""โรงพยาบาลหนองกุงศรี""","""หญิง""","""1992-03-17""",31,"""ไทย""","""ระคายเคืองตาข้างซ้าย เป็นก่อนม…","""ผู้ประกอบวิชาชีพงานพื้นฐานอื่น…",4.0,"""H103""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Unspecified acute conjunctivit…",2024-06-13
977433,"""29-08-2023""","""10""","""มุกดาหาร""","""หนองสูง""","""โนนยาง""","""05806""","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""ชาย""","""1990-08-12""",33,"""ไทย""","""เคืองตามา 2 วัน""","""นักเรียน/นักศึกษา""",1.0,"""H105""","""principal dx""","""กลุ่มโรคตาอักเสบ""","""Blepharoconjunctivitis""",2024-06-13


In [9]:
data_2.write_csv("./output/h10_2023.csv")

In [ ]:
# median_h10 = (
#     combined_h10.select(["opd.diagcode", "opd.date_serv", "province", "district"])
#     .drop_nulls(subset=["province", "district"])
#     .with_columns(
#         week=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%U")
#     )
#     .with_columns(
#         year=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%Y")
#     )
#     .with_columns(
#         pl.when(pl.col("week") == "00")
#         .then(pl.lit("52"))
#         .otherwise(pl.col("week"))
#         .alias("week")
#     )
#     .drop(columns="opd.date_serv")
#     .group_by(["year", "week", "province", "district"], maintain_order=True)
#     .agg(pl.col("opd.diagcode").alias("no_patients").count())
#     .join(
#         total_pop,
#         how="left",
#         left_on=["province", "district", "year"],
#         right_on=["ProvinceThai", "DistrictThaiShort", "year"],
#     )
#     .filter(~pl.col("total_pop").is_null())
#     .with_columns(
#         rate_per_100000=(pl.col("no_patients") / pl.col("total_pop") * 100000)
#     )
#     .drop(columns=["year", "no_patients", "total_pop"])
#     .group_by(by=["week", "province", "district"])
#     .agg(pl.col("rate_per_100000").median().alias("median_rate_per_100000"))
# )